#Develop a regression model to predict the total seed dry weight of plants given species, treatment type, panicle count, and ground cover measurements.”


In [ ]:

from flask import Flask, request, render_template
import pickle
import pandas as pd
import numpy as np

# Load the saved pipeline
with open('final_pipeline.pkl', 'rb') as file:
    model_pipeline = pickle.load(file)

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html', result=None, form_data=None)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get form data
        data = request.form.to_dict()
        print("Form Data:", data)  # Debugging

        # Convert numeric fields to float
        for key in data:
            try:
                data[key] = float(data[key])
            except ValueError:
                pass  # Keep as string for categorical

        # ✅ Compute log columns if raw values exist
        if 'matureseeddw' in data and data['matureseeddw'] > 0:
            data['logmatureseeddw'] = np.log(data['matureseeddw'] + 1)
        if 'immatureseeddw' in data and data['immatureseeddw'] > 0:
            data['logimmatureseeddw'] = np.log(data['immatureseeddw'] + 1)
        if 'matureseeddw' in data and 'immatureseeddw' in data:
            total_seed_dw = data['matureseeddw'] + data['immatureseeddw']
            data['logtotalseeddw'] = np.log(total_seed_dw + 1)

        # ✅ Ensure all expected columns exist
        expected_cols = model_pipeline.named_steps['preprocessor'].feature_names_in_
        for col in expected_cols:
            if col not in data:
                data[col] = 'Unknown' if isinstance(col, str) else 0

        # Convert to DataFrame
        input_df = pd.DataFrame([data])

        # Make prediction
        prediction = model_pipeline.predict(input_df)
        print("Prediction:", prediction)  # Debugging

        return render_template('index.html',
                               result=f"Predicted Total Seed Dry Weight: {prediction[0]:.4f}",
                               form_data=data)
    except Exception as e:
        return render_template('index.html', result=f"Error: {str(e)}", form_data=data)

if __name__ == '__main__':
    app.run(debug=True)


Numeric Columns: ['Year', 'plot', 'trt', 'panicles', 'sqrtpanicles', 'cover0', 'arsincover0', 'cover5', 'arsincover5', 'matureseeddw', 'logmatureseeddw', 'immatureseeddw', 'logimmatureseeddw', 'logtotalseeddw', 'percentimmatureseeds', 'arsinpercentimmatureseeds']
Categorical Columns: ['Species', 'site', 'Treatment']
Preprocessed shape: (102, 27)

Linear Regression Performance
R2: 0.9993074329257371
RMSE: 0.24904436672958044
MAE: 0.18188440572123027

RandomForest Performance
R2: 0.9314316255623256
RMSE: 2.4780366721371414
MAE: 1.0959581153846178

Cross-Validation R2 Scores:
Linear Regression: 0.994974838980468
RandomForest: 0.9865315473744698

Best Params: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}

Pipeline and columns saved successfully!
